In [20]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import lit, mean, col, avg, count
import os
import configparser
from pandas import set_option
set_option('display.max_columns', None)
set_option('display.max_rows', None)


In [2]:
conf = SparkConf()
conf.set('spark.executor.memory', '8g')
conf.set('spark.driver.memory', '8g')
conf.set('spark.executor.cores', '4')
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
spark = SparkSession.builder.appName("eda").config(conf=conf).getOrCreate()

22/08/26 19:19:19 WARN Utils: Your hostname, gustavoalmeida resolves to a loopback address: 127.0.1.1; using 172.21.147.229 instead (on interface eth0)
22/08/26 19:19:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/26 19:19:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
root_path = './data/'

microdados_bronze_read_path = f'{root_path}enem-pyspark-athena-bronze/2020/microdados/MICRODADOS_ENEM_2020.csv'
microdados_silver_write_path = f'{root_path}enem-pyspark-athena-silver/2020/microdados/'

itens_prova_bronze_read_path = f'{root_path}enem-pyspark-athena-bronze/2020/itens_prova/ITENS_PROVA_2020.csv'
itens_prova_silver_write_path = f'{root_path}enem-pyspark-athena-silver/2020/itens_prova/'

itens_prova_gold_write_path = f'{root_path}enem-pyspark-athena-gold/itens_prova/'
microdados_gold_write_path = f'{root_path}enem-pyspark-athena-gold/microdados/'



In [5]:
schema_microdados = StructType([
    StructField("NU_INSCRICAO", IntegerType(), True),
    StructField("NU_ANO", IntegerType(), True),
    StructField("TP_FAIXA_ETARIA", IntegerType(), True),
    StructField("TP_SEXO", StringType(), True),
    StructField("TP_ESTADO_CIVIL", IntegerType(), True),
    StructField("TP_COR_RACA", IntegerType(), True),
    StructField("TP_NACIONALIDADE", IntegerType(), True),
    StructField("TP_ST_CONCLUSAO", IntegerType(), True),
    StructField("TP_ANO_CONCLUIU", IntegerType(), True),
    StructField("TP_ESCOLA", IntegerType(), True),
    StructField("TP_ENSINO", IntegerType(), True),
    StructField("IN_TREINEIRO", IntegerType(), True),
    StructField("CO_MUNICIPIO_ESC", IntegerType(), True),
    StructField("NO_MUNICIPIO_ESC", StringType(), True),
    StructField("CO_UF_ESC", IntegerType(), True),
    StructField("SG_UF_ESC", StringType(), True),
    StructField("TP_DEPENDENCIA_ADM_ESC", IntegerType(), True),
    StructField("TP_LOCALIZACAO_ESC", IntegerType(), True),
    StructField("TP_SIT_FUNC_ESC", IntegerType(), True),
    StructField("CO_MUNICIPIO_PROVA", IntegerType(), True),
    StructField("NO_MUNICIPIO_PROVA", StringType(), True),
    StructField("CO_UF_PROVA", StringType(), True),
    StructField("SG_UF_PROVA", StringType(), True),
    StructField("TP_PRESENCA_CN", IntegerType(), True),
    StructField("TP_PRESENCA_CH", IntegerType(), True),
    StructField("TP_PRESENCA_LC", IntegerType(), True),
    StructField("TP_PRESENCA_MT", IntegerType(), True),
    StructField("CO_PROVA_CN", IntegerType(), True),
    StructField("CO_PROVA_CH", IntegerType(), True),
    StructField("CO_PROVA_LC", IntegerType(), True),
    StructField("CO_PROVA_MT", IntegerType(), True),
    StructField("NU_NOTA_CN", IntegerType(), True),
    StructField("NU_NOTA_CH", IntegerType(), True),
    StructField("NU_NOTA_LC", IntegerType(), True),
    StructField("NU_NOTA_MT", IntegerType(), True),
    StructField("TX_RESPOSTAS_CN", StringType(), True),
    StructField("TX_RESPOSTAS_CH", StringType(), True),
    StructField("TX_RESPOSTAS_LC", StringType(), True),
    StructField("TX_RESPOSTAS_MT", StringType(), True),
    StructField("TP_LINGUA", IntegerType(), True),
    StructField("TX_GABARITO_CN", StringType(), True),
    StructField("TX_GABARITO_CH", StringType(), True),
    StructField("TX_GABARITO_LC", StringType(), True),
    StructField("TX_GABARITO_MT", StringType(), True),
    StructField("TP_STATUS_REDACAO", IntegerType(), True),
    StructField("NU_NOTA_COMP1", IntegerType(), True),
    StructField("NU_NOTA_COMP2", IntegerType(), True),
    StructField("NU_NOTA_COMP3", IntegerType(), True),
    StructField("NU_NOTA_COMP4", IntegerType(), True),
    StructField("NU_NOTA_COMP5", IntegerType(), True),
    StructField("NU_NOTA_REDACAO", IntegerType(), True),
    StructField("Q001", StringType(), True),
    StructField("Q002", StringType(), True),
    StructField("Q003", StringType(), True),
    StructField("Q004", StringType(), True),
    StructField("Q005", StringType(), True),
    StructField("Q006", StringType(), True),
    StructField("Q007", StringType(), True),
    StructField("Q008", StringType(), True),
    StructField("Q009", StringType(), True),
    StructField("Q010", StringType(), True),
    StructField("Q011", StringType(), True),
    StructField("Q012", StringType(), True),
    StructField("Q013", StringType(), True),
    StructField("Q014", StringType(), True),
    StructField("Q015", StringType(), True),
    StructField("Q016", StringType(), True),
    StructField("Q017", StringType(), True),
    StructField("Q018", StringType(), True),
    StructField("Q019", StringType(), True),
    StructField("Q020", StringType(), True),
    StructField("Q021", StringType(), True),
    StructField("Q022", StringType(), True),
    StructField("Q023", StringType(), True),
    StructField("Q024", StringType(), True),
    StructField("Q025", StringType(), True), 
])

In [6]:
df_microdados = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1') \
        .schema(schema_microdados) \
        .csv(microdados_bronze_read_path)

In [7]:
df_microdados.limit(20).toPandas()

22/08/26 19:19:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,NaN,2020,11,F,1,2,1,1,11,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,1501402,Belém,15,PA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,2020,11,M,2,3,1,1,11,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,2408102,Natal,24,RN,1,1,1,1,702.0,689.0,693.0,698.0,NaN,NaN,NaN,NaN,BCBDBDCCCDBDDBADEEACEEAECBECCDBCACDEBEECEBAAE,BCAECABCDCEBDBBBDAABABACBCDDDBDDBABDAADACACBC,99999CADDEDADBAABEAADAAAADCADAABDCCBEECAEECAAC...,EBEBDEDAECBADCADDABACAEDBEEEAEADCCBCEACDACEBB,1,ABBACBCCCDDDDBAEEBECEDACABDCBEDAECDABECCBBADE,BCEECDBCCDEBDBBBBEAAAEACBCDDDBDDBABDABDDBAABC,99999CBDDEDBDBACECABAAAEAECCDCABCBBBBEEDCECAAE...,BBEADECAECBBXCEBADBACAEDACEDADAADCBCCBBDABBEE,1.0,120.0,120.0,120.0,120.0,100.0,580.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,NaN,2020,4,F,2,3,2,2,0,2,1.0,0,2927408.0,Salvador,29.0,BA,2.0,1.0,1.0,2927408,Salvador,29,BA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,C,A,D,3,B,A,B,A,A,A,B,A,B,A,B,A,A,B,A,A,A,A,A,A
3,NaN,2020,2,M,1,3,1,2,0,2,1.0,0,3547304.0,Santana de Parnaíba,35.0,SP,3.0,1.0,1.0,3547304,Santana de Parnaíba,35,SP,1,1,1,1,700.0,688.0,692.0,696.0,NaN,675.0,NaN,NaN,EBEDCCCDCBDBAECAECCECEACCBAABBBCEABCAAAEBEBAE,DABCCACCBCCDCADBDCBAAEBDDBAABDBAEBEBBABDABDBE,DCEAB99999AADAECCEBCDDDCBABDDABBCEEBAEDDDADEBE...,CBDBDCCDDEECBAABBBADEDEDBECDEBBADAEBAABBECBBA,0,BDECCACBEBDEAEDAECCBAEABCCEADDBBCADCBDDEBDBAC,DABCCAECBABECADBDCCABDBCDDAABDBAEBEBBDBDDBDBE,DCEAD99999AADACCCCBBDDCABAECAABBEECBAEDDBBEEBE...,EBDBXCCDAEECBAABABAEDDEDAECBCDCADEEBBABBACBCA,1.0,140.0,200.0,140.0,120.0,160.0,760.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,NaN,2020,4,F,1,3,2,1,1,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,3121605,Diamantina,31,MG,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,G,B,B,3,B,A,B,D,A,A,B,A,B,A,A,A,A,B,A,B,B,A,A,B
5,NaN,2020,3,F,1,3,1,1,1,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,4305207,Cerro Largo,43,RS,1,1,1,1,598.0,567.0,577.0,588.0,NaN,NaN,NaN,NaN,CABBEEACAEBEBCCDACBEDADDCCBECDCBDADDEBDBDACAB,ACEBBDDCADDAACEEAAEBDBDCCDACCDEDCDACADBBBECDD,99999CBECEBEDBEECEABBADCDDADCADAAECDEBAAADEBAD...,BADCAAAEEDBEDBACEDBACCDBABDECCDDCCAEAACEDACCB,1,CEDBDDDCACCBDAEBADCBDCDXAAECEDDDECDDCBDDAEDCC,DABCBDDDAEDBECECBBEDEBDEECBCCCBACDCCADEBDECDD,AAEDCACEEECEAACBAEBBEDACDBAADADDEBBEBAAADEACAA...,AEBCCXCCCACCDABCCABCABCBAECEDBBDBEEEDBBACCDAC,1.0,140.0,120.0,140.0,140.0,160.0,700.0,C,B,C,B,5,C,A,B,E,A,A,B,A,A,A,B,A,A,B,B,A,D,A,B,B
6,NaN,2020,9,M,1,3,1,1,6,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,2611606,Recife,26,PE,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,NaN,2020,4,M,1,1,1,1,

In [8]:
df_microdados.printSchema()

root
 |-- NU_INSCRICAO: integer (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: strin

In [9]:
schema_itens_prova = StructType([
    StructField("CO_POSICAO", IntegerType(), True),
    StructField("SG_AREA", StringType(), True),
    StructField("CO_ITEM", IntegerType(), True),
    StructField("TX_GABARITO", StringType(), True),
    StructField("CO_HABILIDADE", IntegerType(), True),
    StructField("IN_ITEM_ABAN", IntegerType(), True),
    StructField("TX_MOTIVO_ABAN", StringType(), True),
    StructField("NU_PARAM_A", DoubleType(), True),
    StructField("NU_PARAM_B", DoubleType(), True),
    StructField("NU_PARAM_C", DoubleType(), True),
    StructField("TX_COR", StringType(), True),
    StructField("CO_PROVA", IntegerType(), True),
    StructField("TP_LINGUA", IntegerType(), True),
    StructField("IN_ITEM_ADAPTADO", IntegerType(), True),
    StructField("TP_VERSAO_DIGITAL", IntegerType(), True),
])

In [10]:
df_itens_prova = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1') \
        .schema(schema_itens_prova) \
        .csv(itens_prova_bronze_read_path)

In [11]:
df_itens_prova.limit(20).toPandas()

,CO_POSICAO,SG_AREA,CO_ITEM,TX_GABARITO,CO_HABILIDADE,IN_ITEM_ABAN,TX_MOTIVO_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,TX_COR,CO_PROVA,TP_LINGUA,IN_ITEM_ADAPTADO,TP_VERSAO_DIGITAL
0,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA - BRAILE,623,NaN,1,NaN
1,43,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,BRANCA,620,NaN,0,NaN
2,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA,624,NaN,1,NaN
3,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,ROSA,619,NaN,0,NaN
4,41,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AZUL,617,NaN,0,NaN
5,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA - BRAILE,663,NaN,1,NaN
6,41,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AZUL,657,NaN,0,NaN
7,16,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AMARELA,618,NaN,0,NaN
8,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA,664,NaN,1,NaN
9,16,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AMARELA,658,NaN,0,NaN


In [12]:
df_itens_prova.printSchema()

root
 |-- CO_POSICAO: integer (nullable = true)
 |-- SG_AREA: string (nullable = true)
 |-- CO_ITEM: integer (nullable = true)
 |-- TX_GABARITO: string (nullable = true)
 |-- CO_HABILIDADE: integer (nullable = true)
 |-- IN_ITEM_ABAN: integer (nullable = true)
 |-- TX_MOTIVO_ABAN: string (nullable = true)
 |-- NU_PARAM_A: double (nullable = true)
 |-- NU_PARAM_B: double (nullable = true)
 |-- NU_PARAM_C: double (nullable = true)
 |-- TX_COR: string (nullable = true)
 |-- CO_PROVA: integer (nullable = true)
 |-- TP_LINGUA: integer (nullable = true)
 |-- IN_ITEM_ADAPTADO: integer (nullable = true)
 |-- TP_VERSAO_DIGITAL: integer (nullable = true)



In [16]:
df_itens_prova.write.mode('overwrite').parquet(itens_prova_silver_write_path)

In [17]:
df_microdados.write.mode('overwrite').parquet(microdados_silver_write_path)

In [18]:
df_itens_prova = df_itens_prova.withColumn('NU_ANO', lit(2020))

In [19]:
df_itens_prova.limit(20).toPandas()

,CO_POSICAO,SG_AREA,CO_ITEM,TX_GABARITO,CO_HABILIDADE,IN_ITEM_ABAN,TX_MOTIVO_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,TX_COR,CO_PROVA,TP_LINGUA,IN_ITEM_ADAPTADO,TP_VERSAO_DIGITAL,NU_ANO
0,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA - BRAILE,623,NaN,1,NaN,2020
1,43,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,BRANCA,620,NaN,0,NaN,2020
2,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA,624,NaN,1,NaN,2020
3,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,ROSA,619,NaN,0,NaN,2020
4,41,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AZUL,617,NaN,0,NaN,2020
5,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA - BRAILE,663,NaN,1,NaN,2020
6,41,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AZUL,657,NaN,0,NaN,2020
7,16,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AMARELA,618,NaN,0,NaN,2020
8,26,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,LARANJA,664,NaN,1,NaN,2020
9,16,LC,7099,D,15,0,None,1.33337,-0.37357,0.00658,AMARELA,658,NaN,0,NaN,2020


In [20]:
df_itens_prova.write.partitionBy('NU_ANO').mode('overwrite').parquet(itens_prova_gold_write_path)

In [21]:
df_microdados.write.partitionBy('NU_ANO').mode('overwrite').parquet(microdados_gold_write_path)

In [22]:
df_microdados.limit(10).toPandas()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,NaN,2020,11,F,1,2,1,1,11,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,1501402,Belém,15,PA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,2020,11,M,2,3,1,1,11,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,2408102,Natal,24,RN,1,1,1,1,702.0,689.0,693.0,698.0,NaN,NaN,NaN,NaN,BCBDBDCCCDBDDBADEEACEEAECBECCDBCACDEBEECEBAAE,BCAECABCDCEBDBBBDAABABACBCDDDBDDBABDAADACACBC,99999CADDEDADBAABEAADAAAADCADAABDCCBEECAEECAAC...,EBEBDEDAECBADCADDABACAEDBEEEAEADCCBCEACDACEBB,1,ABBACBCCCDDDDBAEEBECEDACABDCBEDAECDABECCBBADE,BCEECDBCCDEBDBBBBEAAAEACBCDDDBDDBABDABDDBAABC,99999CBDDEDBDBACECABAAAEAECCDCABCBBBBEEDCECAAE...,BBEADECAECBBXCEBADBACAEDACEDADAADCBCCBBDABBEE,1.0,120.0,120.0,120.0,120.0,100.0,580.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,NaN,2020,4,F,2,3,2,2,0,2,1.0,0,2927408.0,Salvador,29.0,BA,2.0,1.0,1.0,2927408,Salvador,29,BA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,C,A,D,3,B,A,B,A,A,A,B,A,B,A,B,A,A,B,A,A,A,A,A,A
3,NaN,2020,2,M,1,3,1,2,0,2,1.0,0,3547304.0,Santana de Parnaíba,35.0,SP,3.0,1.0,1.0,3547304,Santana de Parnaíba,35,SP,1,1,1,1,700.0,688.0,692.0,696.0,NaN,675.0,NaN,NaN,EBEDCCCDCBDBAECAECCECEACCBAABBBCEABCAAAEBEBAE,DABCCACCBCCDCADBDCBAAEBDDBAABDBAEBEBBABDABDBE,DCEAB99999AADAECCEBCDDDCBABDDABBCEEBAEDDDADEBE...,CBDBDCCDDEECBAABBBADEDEDBECDEBBADAEBAABBECBBA,0,BDECCACBEBDEAEDAECCBAEABCCEADDBBCADCBDDEBDBAC,DABCCAECBABECADBDCCABDBCDDAABDBAEBEBBDBDDBDBE,DCEAD99999AADACCCCBBDDCABAECAABBEECBAEDDBBEEBE...,EBDBXCCDAEECBAABABAEDDEDAECBCDCADEEBBABBACBCA,1.0,140.0,200.0,140.0,120.0,160.0,760.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,NaN,2020,4,F,1,3,2,1,1,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,3121605,Diamantina,31,MG,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,G,B,B,3,B,A,B,D,A,A,B,A,B,A,A,A,A,B,A,B,B,A,A,B
5,NaN,2020,3,F,1,3,1,1,1,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,4305207,Cerro Largo,43,RS,1,1,1,1,598.0,567.0,577.0,588.0,NaN,NaN,NaN,NaN,CABBEEACAEBEBCCDACBEDADDCCBECDCBDADDEBDBDACAB,ACEBBDDCADDAACEEAAEBDBDCCDACCDEDCDACADBBBECDD,99999CBECEBEDBEECEABBADCDDADCADAAECDEBAAADEBAD...,BADCAAAEEDBEDBACEDBACCDBABDECCDDCCAEAACEDACCB,1,CEDBDDDCACCBDAEBADCBDCDXAAECEDDDECDDCBDDAEDCC,DABCBDDDAEDBECECBBEDEBDEECBCCCBACDCCADEBDECDD,AAEDCACEEECEAACBAEBBEDACDBAADADDEBBEBAAADEACAA...,AEBCCXCCCACCDABCCABCABCBAECEDBBDBEEEDBBACCDAC,1.0,140.0,120.0,140.0,140.0,160.0,700.0,C,B,C,B,5,C,A,B,E,A,A,B,A,A,A,B,A,A,B,B,A,D,A,B,B
6,NaN,2020,9,M,1,3,1,1,6,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,2611606,Recife,26,PE,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,NaN,2020,4,M,1,1,1,1,

Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?

In [23]:
df_microdados.filter((df_microdados['TP_SEXO'] == 'F'))\
                .select(mean('NU_NOTA_MT'))\
                .show()


+-----------------+
|  avg(NU_NOTA_MT)|
+-----------------+
|499.2338038965187|
+-----------------+



Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que estudaram numa escola no estado de São Paulo?

In [24]:
df_microdados.filter(df_microdados['TP_SEXO'] == 'M')\
                .filter(df_microdados['SG_UF_ESC'] == 'SP')\
                .select(mean('NU_NOTA_CH'))\
                .show()


+-----------------+
|  avg(NU_NOTA_CH)|
+-----------------+
|547.6993710691823|
+-----------------+



Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?

In [29]:
df_microdados.filter(col('NO_MUNICIPIO_ESC') == 'Natal')\
                .select(mean(col('NU_NOTA_CH')))\
                .show()

+-----------------+
|  avg(NU_NOTA_CH)|
+-----------------+
|544.5664160401003|
+-----------------+



Qual é o município (município da escola) que obteve a maior nota MÉDIA em matemática?

In [32]:
cidades = ['Biquinhas', 'Novo Xingu', 'Novo Itacolomi', 'Dolcinópolis']
df_microdados.filter(col('NO_MUNICIPIO_ESC').isin(cidades)).toPandas()

In [37]:
df_microdados.groupBy('NO_MUNICIPIO_ESC')\
    .agg(avg('NU_NOTA_MT').alias('media_mt'))\
    .sort('media_mt', ascending=False)\
    .limit(20)\
    .toPandas()

,NO_MUNICIPIO_ESC,media_mt
0,Japorã,937.000000
1,Vargem Grande do Rio Pardo,885.000000
2,Três Arroios,817.000000
3,Alvorada do Sul,791.000000
4,Lagoa Formosa,780.000000
5,Caetanópolis,766.000000
6,Nova Araçá,752.000000
7,Taquaral de Goiás,747.000000
8,Santa Clara d'Oeste,738.000000
9,Potirendaba,728.000000


Quantas pessoas que estudaram numa escola em Recife fizeram a prova do ENEM nessa mesma cidade?

In [39]:
df_microdados.filter((col('NO_MUNICIPIO_ESC') == 'Recife') & (col('NO_MUNICIPIO_PROVA') == 'Recife')).count()

9619

Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?

In [41]:
df_microdados.filter(col('SG_UF_ESC') == 'SC')\
                .filter(col('Q008').isin(['B', 'C', 'D', 'E']))\
                .select(mean(col('NU_NOTA_CH')))\
                .show()

+-----------------+
|  avg(NU_NOTA_CH)|
+-----------------+
|542.4942389326865|
+-----------------+



Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?

In [42]:
df_microdados.filter(col('NO_MUNICIPIO_ESC') == 'Belo Horizonte')\
                .filter(col('TP_SEXO') == 'F')\
                .filter(col('Q002').isin(['F', 'G']))\
                .select(mean(col('NU_NOTA_MT')))\
                .show()

+-----------------+
|  avg(NU_NOTA_MT)|
+-----------------+
|659.4579439252336|
+-----------------+



Quantos alunos não quiseram declarar a cor/raça em 2020 (Entenda a opção “não declarado” nessa pergunta)?

In [14]:
df_microdados.filter(col('TP_COR_RACA') == 0).count()

116883

Qual é o número de alunos do Sexo Feminino que estudaram em escola no estado de São Paulo?


In [17]:
df_microdados.filter(col('TP_SEXO') == 'F')\
                .filter(col('SG_UF_ESC') == 'SP')\
                .count()

91480

Quantos alunos do sexo feminino que estudaram em escola no estado do Rio Grande do Sul possuem EXATAMENTE uma geladeira em casa?

In [18]:
df_microdados.filter(col('SG_UF_ESC') == 'RS')\
                .filter(col('Q012') == 'B')\
                .filter(col('TP_SEXO') == 'F')\
                .count()

18680

Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) em que estudaram mais alunos no ENEM 2020?


In [21]:
df_microdados.groupBy('SG_UF_ESC')\
    .agg(count('NU_ANO').alias('alunos'))\
    .sort('alunos', ascending=False)\
    .limit(20)\
    .toPandas()

,SG_UF_ESC,alunos
0,None,4878540
1,SP,167183
2,CE,104981
3,MG,71614
4,RJ,62583
5,PE,47583
6,BA,47235
7,PR,43060
8,GO,39769
9,RS,36478


Qual é o estado brasileiro (considere a coluna SG_UF_ESC) possui o menor número de alunos cuja mãe possui ensino superior completo?

In [23]:
df_microdados.filter(col('Q002').isin(['F', 'G']))\
    .groupBy('SG_UF_ESC')\
    .agg(count('NU_ANO').alias('alunos'))\
    .sort('alunos', ascending=True)\
    .limit(20)\
    .toPandas()

,SG_UF_ESC,alunos
0,RR,795
1,AP,930
2,AC,932
3,SE,1824
4,TO,2013
5,RO,2071
6,AL,2553
7,PI,3195
8,RN,3225
9,MS,3583


Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de pessoas na faixa “entre 26 e 30 anos”?

In [25]:
df_microdados.filter(col('TP_FAIXA_ETARIA') == 11)\
    .groupBy('SG_UF_ESC')\
    .agg(count('NU_ANO').alias('alunos'))\
    .sort('alunos', ascending=False)\
    .limit(20)\
    .toPandas()

,SG_UF_ESC,alunos
0,None,567071
1,CE,1880
2,AM,828
3,SP,580
4,GO,518
5,PA,479
6,PE,461
7,RJ,456
8,BA,450
9,MG,432


Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o TERCEIRO maior número de alunos cuja residência possui PELO MENOS 2 banheiros?


In [26]:
df_microdados.filter(col('Q008').isin(['C', 'D', 'E']))\
    .groupBy('SG_UF_ESC')\
    .agg(count('NU_ANO').alias('alunos'))\
    .sort('alunos', ascending=False)\
    .limit(20)\
    .toPandas()

,SG_UF_ESC,alunos
0,None,1151573
1,SP,83502
2,MG,30912
3,RJ,24367
4,PR,20938
5,CE,15533
6,GO,14562
7,RS,14470
8,SC,13104
9,BA,13057


Quantos alunos do sexo feminino se autodeclararam pretos?

In [27]:
df_microdados.filter(col('TP_COR_RACA') == 2)\
                .filter(col('TP_SEXO') == 'F')\
                .count()

444346

Quantos alunos estrangeiros fizeram o ENEM 2020?

In [28]:
df_microdados.filter(col('TP_NACIONALIDADE') == 3)\
                .count()

8036

Qual é a diferença da nota média em matemática dos alunos que estudaram o ensino médio em escola pública e em escola privada?

In [35]:
privada = df_microdados.filter(col('TP_ESCOLA') == 3)\
            .select(mean(col('NU_NOTA_MT')).alias('media_mt'))\
            .collect()[0]['media_mt']
            
publica = df_microdados.filter(col('TP_ESCOLA') == 2)\
            .select(mean(col('NU_NOTA_MT')).alias('media_mt'))\
            .collect()[0]['media_mt']
    
    
    
privada - publica

112.76653895145756

Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de alunos do sexo feminino indígenas?

In [36]:
df_microdados.filter(col('TP_COR_RACA') == 5)\
    .filter(col('TP_SEXO') == 'F')\
    .groupBy('SG_UF_ESC')\
    .agg(count('NU_ANO').alias('alunos'))\
    .sort('alunos', ascending=False)\
    .limit(20)\
    .toPandas()

,SG_UF_ESC,alunos
0,None,18081
1,AM,398
2,CE,380
3,SP,298
4,PE,249
5,BA,200
6,MG,144
7,PA,144
8,MS,118
9,RJ,118


Qual é a diferença entre o número de alunos cujo pai possui pós-graduação completa e o número de alunos cuja mãe possui pós-graduação completa?

In [39]:
pai = df_microdados.filter(col('Q001') == 'G')\
            .count()
            
mae = df_microdados.filter(col('Q002') == 'G')\
            .count()
    
    
pai - mae

-194717

Quantos alunos cuja residência possui ATÉ 2 carros estudaram na região NORDESTE do Brasil?

In [40]:
df_microdados.filter(col('Q010').isin(['B', 'C']))\
            .filter(col('SG_UF_ESC').isin(['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']))\
            .count()

74533

Quantos alunos que estudaram em escolas em zona rural possuem internet em casa?

In [41]:
df_microdados.filter(col('TP_LOCALIZACAO_ESC') == 2)\
            .filter(col('Q025') == 'B')\
            .count()

18979

Qual é o nome do município que contém a SEGUNDA maior quantidade total de inscritos no ENEM 2020?

In [42]:
df_microdados.groupBy('NO_MUNICIPIO_ESC')\
    .agg(count('NU_ANO').alias('alunos'))\
    .sort('alunos', ascending=False)\
    .limit(20)\
    .toPandas()

,NO_MUNICIPIO_ESC,alunos
0,None,4878540
1,São Paulo,38277
2,Fortaleza,27783
3,Rio de Janeiro,26431
4,Brasília,19840
5,Manaus,17120
6,Belo Horizonte,11826
7,Recife,10735
8,Salvador,10500
9,Curitiba,9868
